In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import utils as uti

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
import matplotlib
print(matplotlib.__version__)
print(cartopy.__version__)

In [ ]:


#A = uti.MakeDict4Exp( exp='fmthist_MM_control_TEM' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )
#A = uti.MakeDict4Exp( exp='fmthist_CAM5_x01' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )
#A = uti.MakeDict4Exp( exp='fmthist_CAM5_RDG_BLJ_xhigh' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )
#A = uti.MakeDict4Exp( exp='fmthist_CAM5_RDG_BLJ_xhigh' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )
#B = uti.MakeDict4Exp( exp='fmthist_CAM5_RDG_BLJ_zero' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )
#B = uti.MakeDict4Exp( exp='fmthist_MM_x21_2' , user='juliob', subd='regridded' , hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )


A = uti.MakeDict4Exp( exp='fmthist_MM_control_TEM_2' , user='juliob', subd='regridded' , 
                     hsPat='cam.h0' , ymdPat='*' ,verbose=True, open_dataset=True )


#A = uti.MakeDict4Exp( exp='cDEV_ne120pg3_F2000_L93_noFGW' , user='juliob', subd='regridded' , 
#                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )

B = uti.MakeDict4Exp( exp='cDEV_ne240pg3_FMTHIST_aicn_x02' , user='juliob', subd='regridded_1x1' , 
                    hsPat='cam.h0a' , ymdPat='2004-*' ,verbose=True, open_dataset=True )
#B = uti.MakeDict4Exp( exp='cDEV_ne120pg3_F2000_L93_noFGW' , user='juliob', subd='regridded' , 
#                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )



#uti.trim_to_year( A , nyr_max=1000 )
#uti.trim_to_year( B , nyr_max=1000 )
Dc = A.X
Dx = B.X

exp_C=A.exp
exp_X=B.exp
exp_Val='ERA5'

In [ ]:
########################
# Some massaging
########################

##
#ps_c =Dc.PS.values
lon_c=Dc.lon.values
lat_c=Dc.lat.values
lev_c=Dc.lev.values
zlev_c=-7.0*np.log( lev_c/1_000. )

hyai_c=Dc.hyai.values
hybi_c=Dc.hybi.values
hyam_c=Dc.hyam.values
hybm_c=Dc.hybm.values

#ps_x =Dx.PS.values
lon_x=Dx.lon.values
lat_x=Dx.lat.values
lev_x=Dx.lev.values
zlev_x=-7.0*np.log( lev_x/1_000. )


In [ ]:
season='nov'
UUc_s,years_c,months_c = Av.Seasonal( ds=Dc, season=season , fld='U',return_time=True)
UUx_s,years_x,months_x = Av.Seasonal( ds=Dx, season=season , fld='U',return_time=True)

UUc_sz=np.average( UUc_s , axis=2 )
UUx_sz=np.average( UUx_s , axis=2 )

In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False

Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon']

UUv_sz=np.average( UUv_s , axis=2 )

In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=np.linspace( -60,140,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'

Axes1 = Pu.axes_def(n=1,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,UUc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,UUc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_x,zlev_x,UUx_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_x,zlev_x,UUx_sz ,levels=clev, colors='black')
ax1.set_title(f"Test <{exp_X}> {season.upper()} {years_x[0]}-{years_x[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


Axes1 = Pu.axes_def(n=3,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
